In [ ]:
import math

from math import exp
import numpy as np

In [ ]:
k1 = k3 = k5 = 0

x2 = x4 = 0

w2 = w4 = 0

i = 1j

phi1 = phi2 = phi3 = phi4 = phi5 = 0

In [ ]:
me = 9.1093837e-31  # Kg
hbar = 1.054571817e-34  # Js

def k(E, V, m=me):
    return math.sqrt(2 * m * (E - V)) / hbar

def x(E, V, m=me):
    return math.sqrt(2 * m * (V - E)) / hbar

In [ ]:

K = exp(x2 * w2 + x4 * w4) \
    * (exp(i * (- phi1 + phi2 + phi3 + phi4 + phi5)) - exp(i * (phi1 + phi2 - phi3 - phi4 + phi5))) \
    + exp(x2 * w2 - x4 * w4) \
    * (- exp(i * (- phi1 + phi2 + phi3 - phi4 - phi5)) + exp(i * (phi1 + phi2 - phi3 + phi4 - phi5))) \
    + exp(- x2 * w2 + x4 * w4) \
    * (- exp(i * (- phi1 - phi2 - phi3 + phi4 + phi5)) + exp(i * (phi1 - phi2 + phi3 - phi4 + phi5))) \
    + exp(- x2 * w2 - x4 * w4) \
    * (- exp(i * (- phi1 - phi2 - phi3 - phi4 - phi5)) + exp(i * (phi1 - phi2 + phi3 + phi4 - phi5)))

Kconj = np.conj

Ksq = 0

In [ ]:
T = ((2 ** 8) * k1 * ((x2 * k3 * x4) ** 2 ) * k5 ) / (Ksq * (k1**2 + x2**2) * (k3**2 + x2**2) * (k3**2 + x4**2) * (k5**2 + x4**2))